In [1]:
import pandas as pd
import numpy as np
import scprep

import scVGAE

## Read data

In [2]:
df = pd.read_csv('data/sample_data.csv.gz', index_col=0)
df

,A1BG,AAAS,AAK1,AAMDC,AAMP,AASDHPPT,AATF,ABCB10,ABCE1,ABCF1,...,ZNHIT1,ZNHIT3,ZNRD1,ZPR1,ZRANB2,ZRSR2,ZSCAN16-AS1,ZSWIM7,ZWINT,ZYX
NK,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Mono/DC,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,0
NK,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Progenitor cells,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Progenitor cells,0,0,0,0,1,2,2,2,1,0,...,2,3,2,1,2,0,0,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B cell,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
T cell,0,0,2,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
B cell,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Progenitor cells,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


## Preprocess for imputation

In [3]:
tmp = np.sign(df)
cols = (np.sum(tmp) > int((df.shape[0])*0.05))
rows = (np.sum(tmp, axis=1) > int((df.shape[1])*0.05))
df = np.log(df.loc[rows, cols] + 1)
df = scprep.normalize.library_size_normalize(df)    
df = scprep.transform.sqrt(df)
df

,A1BG,AAAS,AAK1,AAMDC,AAMP,AASDHPPT,AATF,ABCB10,ABCE1,ABCF1,...,ZNHIT1,ZNHIT3,ZNRD1,ZPR1,ZRANB2,ZRSR2,ZSCAN16-AS1,ZSWIM7,ZWINT,ZYX
NK,0.000000,0.000000,3.748314,0.000000,0.000000,0.000000,0.000000,0.000000,3.748314,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
Mono/DC,2.856121,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.856121,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,3.595722,0.000000,0.0
NK,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.838370,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
Progenitor cells,0.000000,0.000000,0.000000,0.000000,1.303089,1.640528,1.640528,1.640528,1.303089,0.000000,...,1.640528,1.842847,1.640528,1.303089,1.640528,0.0,0.000000,1.303089,1.640528,0.0
Progenitor cells,1.270841,1.599929,0.000000,1.270841,0.000000,1.599929,1.270841,1.797241,0.000000,2.043235,...,2.316255,0.000000,1.270841,1.270841,1.797241,0.0,1.599929,1.270841,2.201161,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B cell,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
T cell,0.000000,0.000000,4.434093,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,3.522048,0.000000,0.0,0.000000,0.000000,0.000000,0.0
B cell,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,3.024141,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
Progenitor cells,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.702154,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.702154,0.0


## Run model

In [4]:
%%time
imputed_matrix = scVGAE.run_model(df, verbose=True)
imputed_matrix

100%|██████████| 100/100 [00:04<00:00, 22.43it/s]

CPU times: user 9.65 s, sys: 3.36 s, total: 13 s
Wall time: 6.98 s


array([[-1.42876327e-01, -3.95619005e-01,  4.74320889e+00, ...,
         4.54064220e-01, -5.55054843e-02, -3.50726247e-01],
       [ 4.76682568e+00, -1.89052552e-01, -3.51845026e-02, ...,
         6.84076118e+00,  7.16158748e-03, -3.75089467e-01],
       [-3.21896523e-01,  1.59435064e-01, -5.12186646e-01, ...,
         1.20696247e-01,  3.05001497e-01, -1.09098375e-01],
       ...,
       [-4.25778925e-02, -2.69131035e-01, -1.96413070e-01, ...,
        -3.73974323e-01, -6.86166286e-02, -1.10217571e-01],
       [-2.01111436e-01, -3.12480867e-01, -1.85364842e-01, ...,
        -2.40649611e-01,  5.75515699e+00, -5.79522491e-01],
       [ 6.79618120e-03,  7.50229359e-02, -3.27238292e-01, ...,
        -1.40128165e-01,  3.02806884e-01, -6.16965592e-02]], dtype=float32)